## This script processes documents, segments them into pages, calculates the occurrences of certain user-defined keywords, highlights these words with different colours and annotates the document with the page numbers where the keyphrases occur.

In [ ]:
import os
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import docx2txt
import re
import pdfplumber
import textract
import ocrmypdf
import pluggy
from tqdm import tqdm
import time
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from statistics import mean
from statistics import stdev
from decimal import *
import requests
from bs4 import BeautifulSoup
import textract
import urllib.request, urllib.error, urllib.parse
import ocrmypdf
import pluggy
import pdfplumber
from tqdm import tqdm
from io import BytesIO
from docx2pdf import convert
import pdfkit
import PyPDF2
from PyPDF2 import PdfFileReader
import fitz
import stamper
from PDFNetPython3 import *
import seaborn as sns
import textwrap
import sys
from datetime import datetime
import codecs
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import datefinder
import datetime
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from textwrap import wrap
from itertools import chain

#import custom functions
from segment_pdf import segment_pdf
from segment_word import segment_word
from segment_html import segment_html

pd.options.display.max_rows = 1000
pd.options.mode.chained_assignment = None

In [ ]:
base = '/Users/Viktoria/Desktop/NLP_AnnotatePdf'
data_repo = os.path.join(base, 'Raw_data')

#scraper = os.path.join(base, 'Web_scraping')
#main = base
#results = os.path.join(base, 'Results')
#library = os.path.join(base, 'Document_library')
#models = os.path.join(base, 'Models')
os.chdir(base)

In [ ]:
#pip install -r requirements.txt

In [ ]:
#pip install git+https://github.com/jbarlow83/OCRmyPDF.git

### Step 1. Access the raw data ('Source_List.csv'). This is a list of titles and URLs to be scraped.

In [ ]:
#This is a messy table where text data and URLs are mixed. Access everything and bring them to a clean format.
os.chdir(data_repo)

df = pd.read_csv('Source_List.csv')

print('N = ', len(df))
df.head()

In [ ]:
#Create a tidy title. Special characters in the filename will break the download later.

def prettify_title(title):
    
    #Create a nice and tidy title. Special characters in the title will throw errors.
    name = re.sub(r'\W+', ' ', title)
    name = re.sub(r'.pdf', '', name)
    name = re.sub('^\s*', '', name)
    name = re.sub('\s*$', '', name)
    name = re.sub('[\[\]:?!-/\+=&)(\"\'\*,]', '', name)
    name = name.strip()
    
    return name

In [ ]:
df['Document']=df['Document'].apply(prettify_title)